In [1]:
import re
import io
import pandas
import data_algebra.cdata
import data_algebra.cdata_impl

In [2]:
buf = io.StringIO(re.sub('[[ \\t]+', '', """
Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,id
5.1,3.5,1.4,0.2,setosa,0
4.9,3.0,1.4,0.2,setosa,1
4.7,3.2,1.3,0.2,setosa,2
"""))
iris = pandas.read_csv(buf)
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,id
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,2


In [3]:
# from:
#   https://github.com/WinVector/cdata/blob/master/vignettes/control_table_keys.Rmd

control_table = pandas.DataFrame({
    'Part': ["Sepal", "Sepal", "Petal", "Petal"],
    'Measure': ["Length", "Width", "Length", "Width"],
    'Value': ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]
})
record_spec = data_algebra.cdata.RecordSpecification(
    control_table,
    control_table_keys = ['Part', 'Measure'],
    record_keys = ['id', 'Species']
    )
record_spec

RecordSpecification
   record_keys: ['id', 'Species']
   control_table_keys: ['Part', 'Measure']
   control_table:
       Part Measure         Value
   0  Sepal  Length  Sepal.Length
   1  Sepal   Width   Sepal.Width
   2  Petal  Length  Petal.Length
   3  Petal   Width   Petal.Width

In [4]:
mp_to_blocks = data_algebra.cdata_impl.RecordMap(blocks_out=record_spec)
print(str(mp_to_blocks))
arranged_blocks = mp_to_blocks.transform(iris)
arranged_blocks

Transform row records of the form:
  record_keys: ['id', 'Species']
 ['id', 'Species', 'Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']
to block records of structure:
RecordSpecification
   record_keys: ['id', 'Species']
   control_table_keys: ['Part', 'Measure']
   control_table:
       Part Measure         Value
   0  Sepal  Length  Sepal.Length
   1  Sepal   Width   Sepal.Width
   2  Petal  Length  Petal.Length
   3  Petal   Width   Petal.Width



,id,Species,Part,Measure,Value
0,0,setosa,Petal,Length,1.4
1,0,setosa,Petal,Width,0.2
2,0,setosa,Sepal,Length,5.1
3,0,setosa,Sepal,Width,3.5
4,1,setosa,Petal,Length,1.4
5,1,setosa,Petal,Width,0.2
6,1,setosa,Sepal,Length,4.9
7,1,setosa,Sepal,Width,3.0
8,2,setosa,Petal,Length,1.3
9,2,setosa,Petal,Width,0.2


In [5]:
mp_to_rows = data_algebra.cdata_impl.RecordMap(blocks_in=record_spec)
print(str(mp_to_rows))
arranged_rows = mp_to_rows.transform(arranged_blocks)
arranged_rows


Transform block records of structure:
RecordSpecification
   record_keys: ['id', 'Species']
   control_table_keys: ['Part', 'Measure']
   control_table:
       Part Measure         Value
   0  Sepal  Length  Sepal.Length
   1  Sepal   Width   Sepal.Width
   2  Petal  Length  Petal.Length
   3  Petal   Width   Petal.Width
to row records of the form:
  record_keys: ['id', 'Species']
 ['id', 'Species', 'Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']



,id,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,0,setosa,5.1,3.5,1.4,0.2
1,1,setosa,4.9,3.0,1.4,0.2
2,2,setosa,4.7,3.2,1.3,0.2
